# Beta Calibration for Policy Impact Analysis
Estimating how Fed funds rate and money supply changes affect inflation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Import our modules
import sys
sys.path.append('..')
from src.config import PROCESSED_DATA_DIR

print("📚 Libraries loaded successfully!")

## 1. Load and Prepare Data

In [ ]:
# Load feature table
df = pd.read_csv('../data/processed/feature_table.csv', index_col=0, parse_dates=True)

# Focus on post-1985 data for more stable monetary policy regime
df_analysis = df[df.index >= '1985-01-01'].copy()

print(f"📊 Full dataset: {len(df)} observations")
print(f"📊 Post-1985 dataset: {len(df_analysis)} observations")
print(f"📅 Analysis period: {df_analysis.index.min().date()} to {df_analysis.index.max().date()}")

# Key variables for analysis
key_vars = {
    'inflation': 'CPIAUCSL_yoy',
    'fed_funds': 'FEDFUNDS', 
    'money_supply': 'M2SL_yoy',
    'unemployment': 'UNRATE',
    'oil_price': 'DCOILWTICO'
}

print(f"\n🎯 Key variables for analysis:")
for name, var in key_vars.items():
    if var in df_analysis.columns:
        recent_val = df_analysis[var].dropna().iloc[-1]
        print(f"   {name}: {recent_val:.2f}")
    else:
        print(f"   {name}: Variable {var} not found")

## 2. Create Lagged Variables for Policy Impact

In [ ]:
# Create analysis dataframe with lagged policy variables
analysis_data = pd.DataFrame(index=df_analysis.index)

# Target variable: inflation rate
analysis_data['inflation'] = df_analysis['CPIAUCSL_yoy']

# Policy variables with lags (monetary policy takes time to affect inflation)
lags = [1, 3, 6, 12]  # 1, 3, 6, 12 months

for lag in lags:
    # Fed funds rate changes
    analysis_data[f'fed_funds_lag{lag}'] = df_analysis['FEDFUNDS'].shift(lag)
    analysis_data[f'fed_funds_change_lag{lag}'] = df_analysis['FEDFUNDS'].diff().shift(lag)
    
    # Money supply growth
    analysis_data[f'm2_growth_lag{lag}'] = df_analysis['M2SL_yoy'].shift(lag)
    
    # Control variables
    analysis_data[f'unemployment_lag{lag}'] = df_analysis['UNRATE'].shift(lag)

# Oil price changes (supply shocks)
if 'DCOILWTICO' in df_analysis.columns:
    analysis_data['oil_price_change'] = df_analysis['DCOILWTICO'].pct_change(12) * 100

# Drop rows with missing values
analysis_data_clean = analysis_data.dropna()

print(f"📊 Analysis dataset: {len(analysis_data_clean)} complete observations")
print(f"📅 Period: {analysis_data_clean.index.min().date()} to {analysis_data_clean.index.max().date()}")
print(f"📈 Variables: {len(analysis_data_clean.columns)} features")

## 3. Estimate Core Policy Beta Coefficients

In [ ]:
# Model 1: Fed Funds Rate Impact on Inflation
print("🏦 ESTIMATING FED FUNDS RATE IMPACT ON INFLATION")
print("=" * 60)

# Prepare features for Fed funds model
fed_features = [
    'fed_funds_lag3', 'fed_funds_lag6', 'fed_funds_lag12',
    'unemployment_lag3', 'unemployment_lag6'
]

if 'oil_price_change' in analysis_data_clean.columns:
    fed_features.append('oil_price_change')

# Check which features are available
available_fed_features = [f for f in fed_features if f in analysis_data_clean.columns]
print(f"Using features: {available_fed_features}")

X_fed = analysis_data_clean[available_fed_features]
y = analysis_data_clean['inflation']

# Remove any remaining NaN values
mask = ~(X_fed.isnull().any(axis=1) | y.isnull())
X_fed_clean = X_fed[mask]
y_clean = y[mask]

if len(X_fed_clean) > 0:
    # Fit OLS regression
    X_fed_const = sm.add_constant(X_fed_clean)
    fed_model = sm.OLS(y_clean, X_fed_const).fit()
    
    print(f"\n📊 Fed Funds Rate Model Results (n={len(X_fed_clean)}):")
    print(f"   R-squared: {fed_model.rsquared:.3f}")
    print(f"   Adjusted R-squared: {fed_model.rsquared_adj:.3f}")
    
    # Extract key coefficients
    fed_coeffs = {}
    print(f"\n📈 Key Coefficients:")
    for var in available_fed_features:
        if var in fed_model.params.index:
            coeff = fed_model.params[var]
            pval = fed_model.pvalues[var]
            fed_coeffs[var] = coeff
            significance = "***" if pval < 0.01 else "**" if pval < 0.05 else "*" if pval < 0.1 else ""
            print(f"   {var}: {coeff:.4f} {significance} (p={pval:.3f})")
    
    # Calculate average Fed funds impact
    fed_impact_coeffs = [fed_coeffs.get(f'fed_funds_lag{lag}', 0) for lag in [3, 6, 12]]
    avg_fed_impact = np.mean([c for c in fed_impact_coeffs if c != 0])
    
    print(f"\n🎯 Average Fed Funds Impact: {avg_fed_impact:.4f}")
    print(f"   → 1% increase in Fed funds rate → {avg_fed_impact:.3f}% change in inflation")
    
else:
    print("❌ Insufficient data for Fed funds model")
    avg_fed_impact = -0.3  # Default assumption

In [ ]:
# Model 2: Money Supply Impact on Inflation
print("\n💰 ESTIMATING MONEY SUPPLY IMPACT ON INFLATION")
print("=" * 60)

# Prepare features for M2 model
m2_features = [
    'm2_growth_lag3', 'm2_growth_lag6', 'm2_growth_lag12',
    'unemployment_lag6'
]

if 'oil_price_change' in analysis_data_clean.columns:
    m2_features.append('oil_price_change')

# Check which features are available
available_m2_features = [f for f in m2_features if f in analysis_data_clean.columns]
print(f"Using features: {available_m2_features}")

X_m2 = analysis_data_clean[available_m2_features]

# Remove any remaining NaN values
mask = ~(X_m2.isnull().any(axis=1) | y.isnull())
X_m2_clean = X_m2[mask]
y_m2_clean = y[mask]

if len(X_m2_clean) > 0:
    # Fit OLS regression
    X_m2_const = sm.add_constant(X_m2_clean)
    m2_model = sm.OLS(y_m2_clean, X_m2_const).fit()
    
    print(f"\n📊 Money Supply Model Results (n={len(X_m2_clean)}):")
    print(f"   R-squared: {m2_model.rsquared:.3f}")
    print(f"   Adjusted R-squared: {m2_model.rsquared_adj:.3f}")
    
    # Extract key coefficients
    m2_coeffs = {}
    print(f"\n📈 Key Coefficients:")
    for var in available_m2_features:
        if var in m2_model.params.index:
            coeff = m2_model.params[var]
            pval = m2_model.pvalues[var]
            m2_coeffs[var] = coeff
            significance = "***" if pval < 0.01 else "**" if pval < 0.05 else "*" if pval < 0.1 else ""
            print(f"   {var}: {coeff:.4f} {significance} (p={pval:.3f})")
    
    # Calculate average M2 impact
    m2_impact_coeffs = [m2_coeffs.get(f'm2_growth_lag{lag}', 0) for lag in [3, 6, 12]]
    avg_m2_impact = np.mean([c for c in m2_impact_coeffs if c != 0])
    
    print(f"\n🎯 Average M2 Growth Impact: {avg_m2_impact:.4f}")
    print(f"   → 1% increase in M2 growth → {avg_m2_impact:.3f}% change in inflation")
    
else:
    print("❌ Insufficient data for M2 model")
    avg_m2_impact = 0.2  # Default assumption

## 4. Validate Model Relationships

In [ ]:
# Visualize key relationships
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        'Fed Funds Rate vs Future Inflation (12m lag)',
        'M2 Growth vs Future Inflation (12m lag)', 
        'Fed Funds Changes vs Inflation Changes',
        'Unemployment vs Inflation (Phillips Curve)'
    ],
    specs=[[{}, {}], [{}, {}]]
)

# Plot 1: Fed funds vs future inflation
if 'fed_funds_lag12' in analysis_data_clean.columns:
    mask = ~(analysis_data_clean['fed_funds_lag12'].isnull() | analysis_data_clean['inflation'].isnull())
    x_data = analysis_data_clean.loc[mask, 'fed_funds_lag12']
    y_data = analysis_data_clean.loc[mask, 'inflation']
    
    fig.add_trace(go.Scatter(
        x=x_data, y=y_data,
        mode='markers', name='Fed Funds vs Inflation',
        marker=dict(color='blue', size=4, opacity=0.6)
    ), row=1, col=1)
    
    # Add trend line
    if len(x_data) > 10:
        z = np.polyfit(x_data, y_data, 1)
        p = np.poly1d(z)
        fig.add_trace(go.Scatter(
            x=x_data, y=p(x_data),
            mode='lines', name='Trend',
            line=dict(color='red', width=2)
        ), row=1, col=1)

# Plot 2: M2 growth vs future inflation  
if 'm2_growth_lag12' in analysis_data_clean.columns:
    mask = ~(analysis_data_clean['m2_growth_lag12'].isnull() | analysis_data_clean['inflation'].isnull())
    x_data = analysis_data_clean.loc[mask, 'm2_growth_lag12']
    y_data = analysis_data_clean.loc[mask, 'inflation']
    
    fig.add_trace(go.Scatter(
        x=x_data, y=y_data,
        mode='markers', name='M2 vs Inflation',
        marker=dict(color='green', size=4, opacity=0.6)
    ), row=1, col=2)

# Plot 3: Fed funds changes vs inflation changes
if 'fed_funds_change_lag6' in analysis_data_clean.columns:
    mask = ~(analysis_data_clean['fed_funds_change_lag6'].isnull() | analysis_data_clean['inflation'].isnull())
    x_data = analysis_data_clean.loc[mask, 'fed_funds_change_lag6']
    y_data = analysis_data_clean.loc[mask, 'inflation']
    
    fig.add_trace(go.Scatter(
        x=x_data, y=y_data,
        mode='markers', name='Fed Changes vs Inflation',
        marker=dict(color='purple', size=4, opacity=0.6)
    ), row=2, col=1)

# Plot 4: Phillips curve (unemployment vs inflation)
if 'unemployment_lag6' in analysis_data_clean.columns:
    mask = ~(analysis_data_clean['unemployment_lag6'].isnull() | analysis_data_clean['inflation'].isnull())
    x_data = analysis_data_clean.loc[mask, 'unemployment_lag6']
    y_data = analysis_data_clean.loc[mask, 'inflation']
    
    fig.add_trace(go.Scatter(
        x=x_data, y=y_data,
        mode='markers', name='Phillips Curve',
        marker=dict(color='orange', size=4, opacity=0.6)
    ), row=2, col=2)

fig.update_layout(
    title='📊 Economic Relationships Validation',
    height=600,
    showlegend=False
)

fig.show()

## 5. Final Beta Coefficients for Simulator

In [ ]:
# Compile final beta coefficients for the simulator
print("🎯 FINAL BETA COEFFICIENTS FOR POLICY SIMULATOR")
print("=" * 60)

# Define the beta coefficients based on our analysis
beta_coefficients = {
    # Fed funds rate impact on inflation (with lag structure)
    'fed_funds_beta': {
        'lag_3': avg_fed_impact * 0.3,   # 30% impact at 3 months
        'lag_6': avg_fed_impact * 0.5,   # 50% impact at 6 months  
        'lag_12': avg_fed_impact * 0.7,  # 70% impact at 12 months
        'total': avg_fed_impact
    },
    
    # Money supply growth impact on inflation
    'money_supply_beta': {
        'lag_3': avg_m2_impact * 0.2,    # 20% impact at 3 months
        'lag_6': avg_m2_impact * 0.4,    # 40% impact at 6 months
        'lag_12': avg_m2_impact * 0.6,   # 60% impact at 12 months
        'total': avg_m2_impact
    },
    
    # Fiscal policy (government spending) - theoretical estimates
    'fiscal_beta': {
        'lag_3': 0.1,    # Fiscal multiplier effects
        'lag_6': 0.15,   
        'lag_12': 0.08,  
        'total': 0.15
    },
    
    # Analysis metadata
    'metadata': {
        'estimation_period': f"{analysis_data_clean.index.min().date()} to {analysis_data_clean.index.max().date()}",
        'observations': len(analysis_data_clean),
        'fed_model_r2': fed_model.rsquared if 'fed_model' in locals() else 'N/A',
        'm2_model_r2': m2_model.rsquared if 'm2_model' in locals() else 'N/A'
    }
}

print(f"\n📊 Fed Funds Rate Beta:")
for lag, coeff in beta_coefficients['fed_funds_beta'].items():
    if lag != 'total':
        print(f"   {lag}: {coeff:.4f}")
print(f"   TOTAL IMPACT: {beta_coefficients['fed_funds_beta']['total']:.4f}")

print(f"\n💰 Money Supply Beta:")
for lag, coeff in beta_coefficients['money_supply_beta'].items():
    if lag != 'total':
        print(f"   {lag}: {coeff:.4f}")
print(f"   TOTAL IMPACT: {beta_coefficients['money_supply_beta']['total']:.4f}")

print(f"\n🏛️ Fiscal Policy Beta:")
for lag, coeff in beta_coefficients['fiscal_beta'].items():
    if lag != 'total':
        print(f"   {lag}: {coeff:.4f}")
print(f"   TOTAL IMPACT: {beta_coefficients['fiscal_beta']['total']:.4f}")

print(f"\n📋 Analysis Summary:")
print(f"   Period: {beta_coefficients['metadata']['estimation_period']}")
print(f"   Observations: {beta_coefficients['metadata']['observations']}")

# Save coefficients for use in simulator
import json
with open('../data/processed/beta_coefficients.json', 'w') as f:
    json.dump(beta_coefficients, f, indent=2, default=str)

print(f"\n💾 Beta coefficients saved to: ../data/processed/beta_coefficients.json")
print(f"\n✅ Beta calibration complete! Ready for scenario simulation.")